In [219]:
import pandas as pd

In [3]:
df_test = pd.read_csv('coursera_sessions_test.txt', sep = ';', header= None)
df_test.columns = ['viewed', 'bought']
df_train = pd.read_csv('coursera_sessions_train.txt', sep = ';', header= None)
df_train.columns = ['viewed', 'bought']

In [4]:
df_train.head()

,viewed,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


In [5]:
df_train1 = df_train
for i in range(df_train.shape[0]):
    df_train1.loc[i,:][0] = [int(x) for x in df_train1.loc[i,:][0].split(',')]

In [6]:
for i in range(df_train.shape[0]):
    #print i
    #print df_train1.loc[i,:][1]
    if type(df_train1.loc[i,:][1]) == float:
        df_train1.loc[i,:][1] = 0
    elif type(df_train1.loc[i,:][1]) == int:
        df_train1.loc[i,:][1] = int(df_train1.loc[i,:][1])
    else:
        df_train1.loc[i,:][1] = [int(x) for x in df_train1.loc[i,:][1].split(',')]
    #print df_train1.loc[i,:][1]

In [276]:
bought_train = OrderedCounter()
count = 0
for i in df_train1.bought:
    #print i
    if type(i) == int and i != 0:
        if i in bought_train:
            count = bought_train[i]
            bought_train[i] = count + 1
            #print bought_train[i]
            #print '/n'
        else:
            bought_train[i] = 1
            #print bought_train[i]
            #print '/n'
    else:
        if i != 0:
            for item in i:
                #print item
                if item in bought_train:
                    count = bought_train[item]
                    bought_train[item] = count + 1
                    #print item, bought_train[item]
                    #print '/n'
                else:
                    if item != 0:
                        bought_train[item] = 1
                        #print item, bought_train[item]
                        #print '/n'

In [8]:
viewed_train = {}
count = 0
for i in df_train1.viewed:
    for item in i:
        #print item
        if item in viewed_train:
            count = viewed_train[item]
            viewed_train[item] = count + 1
            #print item, viewed_train[item]
            #print '/n'
        else:
            viewed_train[item] = 1

In [257]:
from collections import Counter, OrderedDict

class OrderedCounter(Counter, OrderedDict):
     'Counter that remembers the order elements are first seen'
    def __repr__(self):
         return '%s(%r)' % (self.__class__.__name__,
                            OrderedDict(self))
    def __reduce__(self):
         return self.__class__, (OrderedDict(self),)

In [9]:
import operator
sorted_viewed_train = sorted(viewed_train.items(), key=operator.itemgetter(1), reverse=True)

In [10]:
sorted_viewed_train[:5]

[(73, 677), (158, 641), (204, 396), (262, 387), (162, 318)]

In [96]:
viewed_recom = {}
for i in sorted_viewed_train:
    viewed_recom[i[0]] = i[1]

In [12]:
sorted_bought_train = sorted(bought_train.items(), key=operator.itemgetter(1), reverse=True)

In [95]:
bought_recom = {}
for i in sorted_bought_train:
    bought_recom[i[0]] = i[1]

In [233]:
df_test.iloc[5,:][0]

[43, 44, 43, 45, 43, 45, 43, 46]

In [286]:
import operator
def average_recall(n, testing_list, type_of_recommendation):
    recall_list = []
    #number = 0
    for index, row in testing_list.iterrows():
        #number +=1
        #print number
        #count = 0
        numerator = 0
        denominator = 1
        if row[1] == 0:
            continue
        #строим рекомендации для этого пользователя
        viewed_products = pd.Series(row[0]).unique()
        recommend_dict = OrderedCounter()
        for i in viewed_products:
            #print i
            if i in type_of_recommendation.keys():
                recommend_dict[i] = type_of_recommendation[i] 
            else:
                recommend_dict[i] = 0
        sorted_x = OrderedCounter(recommend_dict).most_common(n)
        recommendations = [x[0] for x in sorted_x][:n]
        #если одна покупка
        if type(row[1]) == int:
            if row[1] in recommendations:
                numerator += 1
                denominator = 1
        # если несколько покупок
        if type(row[1]) == list:
            for item in row[1]:
                if item in recommendations:
                    numerator += 1
            denominator = len(row[1])
        #print float(numerator)/denominator
        recall_list.append(float(numerator)/denominator)
    result = sum(recall_list) / float(len(recall_list))
    return round(result, 2)



In [287]:
import operator
def average_precision(n, testing_list, type_of_recommendation):
    recall_list = []
    for index, row in testing_list.iterrows():
        numerator = 0
        count = 0
        denominator = 1
        if row[1] == 0:
            continue
        #строим рекомендации для этого пользователя
        viewed_products = pd.Series(row[0]).unique()
        recommend_dict = OrderedCounter()
        for i in viewed_products:
            #print i
            if i in type_of_recommendation.keys():
                recommend_dict[i] = type_of_recommendation[i] 
            else:
                recommend_dict[i] = 0
        sorted_x = OrderedCounter(recommend_dict).most_common(n)
        recommendations = [x[0] for x in sorted_x][:n]
        #если одна покупка
        if type(row[1]) == int:
            if row[1] in recommendations:
                numerator += 1
                denominator = n
        # если несколько покупок
        if type(row[1]) == list:
            for item in row[1]:
                if item in recommendations:
                    numerator += 1
            denominator = n
        recall_list.append(float(numerator)/denominator)
    result = sum(recall_list) / float(len(recall_list))
    return round(result, 2)




In [ ]:
for i in viewed_products:
            #print i
            if i in type_of_recommendation.keys():
                recommend_dict[i] = type_of_recommendation[i] 
            else:
                recommend_dict[i] = 0
        sorted_x = sorted(recommend_dict.items(), key=operator.itemgetter(1), reverse=True)
        recommendations = [x[0] for x in sorted_x][:quantity]

#   1

In [168]:
print average_recall(1, df_train, viewed_recom), average_precision(1, df_train, viewed_recom), average_recall(5, df_train, viewed_recom),average_precision(5, df_train, viewed_recom)

0.44 0.51 0.82 0.21


In [139]:
df_test1 = df_test


for i in range(df_test1.shape[0]):
    df_test1.loc[i,:][0] = [int(x) for x in df_test1.loc[i,:][0].split(',')]

In [150]:
df_test1.head()

,viewed,bought
0,"[6, 7, 8]",63
1,"[13, 14, 15]",63
2,"[22, 23]",63
3,"[28, 29, 30, 31, 32, 33]",63
4,"[40, 41]",63


In [163]:
for i in range(df_test1.shape[0]):
    #print i
    #print df_test1.loc[i,:][1]
    if type(df_test1.loc[i,:][1]) == float:
        df_test1.loc[i,:][1] = 0
    elif type(df_test1.loc[i,:][1]) == int:
        df_test1.loc[i,:][1] = int(df_test1.loc[i,:][1])
    elif type(df_test1.loc[i,:][1]) == str:
        df_test1.loc[i,:][1] = [int(x) for x in df_test1.loc[i,:][1].split(',')]
    else:
        df_test1.loc[i,:][1] = [int(x) for x in df_test1.loc[i,:][1]]
    #print df_test1.loc[i,:][1]

### 2

In [195]:
print average_recall(1, df_test1, viewed_recom), average_precision(1, df_test1, viewed_recom), average_recall(5, df_test1, viewed_recom),average_precision(5, df_test1, viewed_recom)

 0.42 0.49 0.8 0.2


### 3

In [225]:
print average_recall(1, df_train, bought_recom), average_precision(1, df_train, bought_recom), average_recall(5, df_train, bought_recom),average_precision(5, df_train, bought_recom)

0.68 0.79 0.93 0.25


In [200]:
print average_recall(1, df_train, bought_recom), average_precision(1, df_train, bought_recom), average_recall(5, df_train, bought_recom),average_precision(5, df_train, bought_recom)

0.68 0.79 0.93 0.25


### 4

In [288]:
print average_recall(1, df_test, bought_train), average_precision(1, df_test, bought_train), average_recall(5, df_test, bought_train),average_precision(5, df_test, bought_train)

0.46 0.53 0.82 0.21
